In [1]:
lines = [line.rstrip('\n') for line in open('data-train.txt')]

In [2]:
import nltk
import string
from nltk.tokenize import RegexpTokenizer
import re


def preprocess(sentence):
    sentence = sentence.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(sentence)
    return tokens

def prepare(text):
    text = nltk.sent_tokenize(text)
    words = []
    for s in text:
        for w in preprocess(s):
            words.append(w)
    return words    

# Подготовка данных
Каждое предложение переводим в нижний регистр и выполняем токенизацию. 

Каждый объект <i>(язык, текст)</i> превращаем в <i>(язык, список слов из текста)</i>

In [3]:
data = []

for i in range(1, len(lines)):
    lang, text = lines[i].split('\t', 1)
    data.append((prepare(text), lang))

## Построение train, validate
Выполняем StratifiedShuffleSplit данных из data-train (с соблюдением пропрорций классов, test_size=0.1)

In [5]:
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit

X = data
y = []
for i in range(len(data)):
    y.append(data[i][1])
    
X = np.array(X)
y = np.array(y)
    
X_train = None
X_test = None
y_train = None
y_test = None
    
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=0)
for train_index, test_index in sss.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

In [6]:
def conv(X):
    X_new = []
    for i in range(len(X)):
        X_new.append((X[i][0], X[i][1]))
    return X_new

X_train = conv(X_train)
X_test = conv(X_test)

(['чего', 'он', 'вообще', 'хотел', 'добиться', 'никто', 'понять', 'не', 'мог', 'он', 'заехал', 'в', 'лес', 'вышел', 'осмотрелся', 'затряс', 'головой', 'снова', 'влез', 'в', 'машину', 'и', 'они', 'поехали', 'дальше', 'то', 'же', 'самое', 'произошло', 'и', 'посреди', 'вспаханного', 'поля', 'и', 'на', 'подвесном', 'мосту', 'и', 'на', 'крыше', 'многоэтажной', 'стоянки', 'папа', 'сошел', 'с', 'ума', 'да', 'скучно', 'спросил', 'дудли', 'у', 'матери', 'вечером', 'того', 'же', 'дня', 'дядя', 'вернон', 'привез', 'их', 'на', 'берег', 'моря', 'запер', 'в', 'машине', 'и', 'исчез'], 'ru')


In [7]:
l_arr = ['be','bg','cs','de','el','en','es','fr','hr','hsb','hy','it','la','lt','lv','mk','nl','pl','pt','ro','ru','sk','sl','sr','sv','uk']

l_ids = {}
l_i = 0
l_from_id = {}

for lang in l_arr:
    l_ids[lang] = l_i
    l_from_id[l_i] = lang
    l_i += 1

In [9]:
from collections import defaultdict
import itertools
import nltk.classify.util, nltk.metrics
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import movie_reviews
from nltk.metrics.scores import precision, recall


def evaluate_classifier(features_extractor, train, test):
    """
         features_extractor - function for extraction features from review. 
         train, test - samples
    """
    train_feats = [(features_extractor(review), sent) for review, sent in train]
    test_feats = [(features_extractor(review), sent) for review, sent in test]
 
    classifier = NaiveBayesClassifier.train(train_feats)
#     refsets = defaultdict(set)
#     testsets = defaultdict(set)
 
#     for i, (feats, label) in enumerate(test_feats):
#         observed = classifier.classify(feats)
 
    print('accuracy:', nltk.classify.util.accuracy(classifier, test_feats))
    classifier.show_most_informative_features()
    return classifier

# Модель 1
Попробуем NaiveBayesClassifier c признаками bag-of-symbols
### Результат на validate:
accuracy: 0.96

In [8]:
def word_feats(words):
    return dict([(sym, True) for word in words for sym in word])

evaluate_classifier(word_feats, X_train, X_test)

accuracy: 0.9649670840325287
Most Informative Features
                       ě = True               cs : ru     = 104004.6 : 1.0
                       ι = True               el : ru     = 103911.3 : 1.0
                       ί = True               el : ru     = 103911.3 : 1.0
                       έ = True               el : ru     = 103911.3 : 1.0
                       κ = True               el : ru     = 103911.3 : 1.0
                       γ = True               el : ru     = 103911.3 : 1.0
                       α = True               el : ru     = 103911.3 : 1.0
                       ε = True               el : ru     = 103911.3 : 1.0
                       ο = True               el : ru     = 103911.3 : 1.0
                       ν = True               el : ru     = 103911.3 : 1.0


# Модель 2
Попробуем NaiveBayesClassifier c признаками bag-of-symbols, и добавим к ним популярные биграмы символов.
### Результат на validate:
accuracy: 0.98
### Результат проверки на kaggle:
0.35437 - значительно хуже результатов триграмной языковой модели.

In [10]:
from nltk.collocations import BigramCollocationFinder


def freq_scorer(n_ii, n_ix_xi_tuple, n_xx):
    return n_ii / n_xx

def bigram_word_feats(words, score_fn=freq_scorer, n=10):
    symbols = [s for w in words for s in w]
    bigram_finder = BigramCollocationFinder.from_words(symbols)
    bigrams = bigram_finder.nbest(score_fn, n)
    return dict([(ngram, True) for ngram in itertools.chain(symbols, bigrams)])
 
clf = evaluate_classifier(bigram_word_feats, X_train, X_test)

accuracy: 0.9899574028656254
Most Informative Features
                       ě = True               cs : ru     = 104004.6 : 1.0
                       ι = True               el : ru     = 103911.3 : 1.0
                       ί = True               el : ru     = 103911.3 : 1.0
                       έ = True               el : ru     = 103911.3 : 1.0
                       ε = True               el : ru     = 103911.3 : 1.0
                       κ = True               el : ru     = 103911.3 : 1.0
                       α = True               el : ru     = 103911.3 : 1.0
                       ο = True               el : ru     = 103911.3 : 1.0
                       γ = True               el : ru     = 103911.3 : 1.0
                       ν = True               el : ru     = 103911.3 : 1.0


In [11]:
lines = [line.rstrip('\n') for line in open('data-test.txt')]

In [20]:
import csv


test_out = open('test.out', 'w')
writer = csv.writer(test_out)
writer.writerow(['id', 'be','bg','cs','de','el','en','es','fr','hr','hsb','hy','it','la','lt','lv','mk','nl','pl','pt','ro','ru','sk','sl','sr','sv','uk'])

l_count = len(l_arr)

for i in range(1, len(lines)):
    k, text = lines[i].split('\t', 1)
    text = prepare(text)

    probs = [0] * l_count
    p = clf.classify(bigram_word_feats(text))
    probs[l_ids[p]] = 1
    probs = [k] + probs
    writer.writerow(probs)

test_out.flush()
test_out.close()